In [3]:
import os

import matplotlib.pyplot as plt
import numpy as np

from pyaml.accelerator import Accelerator
from pyaml.common.constants import ACTION_MEASURE
from pyaml.configuration.factory import Factory
from pyaml.magnet.magnet import Magnet

In [4]:
sr = Accelerator.load("p.yaml")
# This is a fix to be able to load multiple times.
# This will be fixed after pyAML workshop
Factory.clear()
# sr.design.get_lattice().disable_6d()

30 Jan% 2026, 17:48:28 | WARNING | PyAML Tango control system binding (0.3.3) initialized with name 'live' and TANGO_HOST=localhost:11000


For the live control mode, you should have some control system emulation runing. It is possible to do 
```
apptainer pull virtual-accelerator.sif oras://gitlab-registry.synchrotron-soleil.fr/software-control-system/containers/apptainer/virtual-accelerator:latest
apptainer run virtual-accelerator.sif
```
this will run SOLEIL II proof-of-concept digital twin on localhost:11000. You can play with the digital twin itself (without pyAML) via jive to check that everything is working. You can run jive in a different terminal with 
```
apptainer pull jive.sif https://gitlab.synchrotron-soleil.fr/api/v4/projects/2739/packages/generic/jive/latest/jive.sif
apptainer run jive.sif
```
On linux you may need additionally to configure X11
```
export DISPLAY=:0
xhost +local:root
```

NOTE: This is just a demonstration of pyAML functionality. Certain things may be done stupidly. The person who wrote this jupyter notebook only cared about showing that the code is working, not about intelligently controlling the accelerator.

In [5]:
control_mode = sr.live
control_mode

TangoControlSystem(name='live', tango_host='localhost:11000', debug_level=None, lazy_devices=True, scalar_aggregator='tango.pyaml.multi_attribute', vector_aggregator=None, timeout_ms=3000)

In [6]:
tune_monitor = control_mode.get_betatron_tune_monitor("BETATRON_TUNE")
print(tune_monitor.tune.get())
tune_monitor

[0.21451629 0.30517208]


BetatronTuneMonitor(peer='TangoControlSystem:live', name='BETATRON_TUNE', lattice_names=None, tune_h=AttributeReadOnly(attribute='simulator/ringsimulator/ringsimulator/Tune_h', unit='', range=None), tune_v=AttributeReadOnly(attribute='simulator/ringsimulator/ringsimulator/Tune_v', unit='', range=None))

In [9]:
qcorrectors = control_mode.get_magnets("QCORR")
first_qcorrector = qcorrectors[0]
print(
    f"The ring has {len(qcorrectors)} quadrupolar correctors. "
    f"The first one is named {first_qcorrector.get_name()} (lattice name)."
)
qcorrectors[0]  # String representation of the first corrector

The ring has 208 quadrupolar correctors. The first one is named QCORR_001 (lattice name).


Quadrupole(peer='TangoControlSystem:live', name='QCORR_001', model_name='QCORR_001', magnet_model=IdentityMagnetModel(powerconverter=None, physics=Attribute(attribute='AN01-AR/EM-COR/CQLN.03/strength', unit='1/m', range=None), unit='1/m'))

In [ ]:
# qcorrectors[0].strength.set(0.2)
qcorrectors[0].strength.get()

### Standard tuning tool for tune correction

In [43]:
tune_correction = control_mode.get_tune_tuning("DEFAULT_TUNE_CORRECTION")

tune_correction

Tune(peer='TangoControlSystem:live', name='DEFAULT_TUNE_CORRECTION', lattice_names=None, quad_array='QCORR', betatron_tune='BETATRON_TUNE', delta=0.001)

Let's try to set the tune using the tune correction tool.

In [44]:
from pyaml import PyAMLException

try:
    tune_correction.set([0.21, 0.31])
except PyAMLException as e:
    print(f"We've got an error saying: {e}")

We've got an error saying: TuneResponse.correct(): no matrix loaded or measured


In [45]:
def tune_callback(step: int, action: int, m: Magnet, dtune: np.array):
    if action == ACTION_MEASURE:
        # On action measure, the measured dq / dk is passed as argument
        print(f"Tune response: #{step} {m.get_name()}")
    return True

It is not possible to do so without measuring the response matrix (or loading it from a file). In the properties of the tune_correction we can find TuneResponse object.

In [46]:
%%time
tune_correction.response.measure(callback=tune_callback, set_wait_time=1.5)

Tune response: #0 QCORR_001 [0. 0.]
Tune response: #1 QCORR_002 [ 0.14108408 -1.12654246]
Tune response: #2 QCORR_003 [ 0.12505253 -0.33585068]
Tune response: #3 QCORR_004 [ 0.12505253 -0.33585068]
Tune response: #4 QCORR_005 [ 0.12510871 -0.33496161]
Tune response: #5 QCORR_006 [ 0.12180106 -0.33812008]
Tune response: #6 QCORR_007 [ 0.12180106 -0.33812008]
Tune response: #7 QCORR_008 [ 0.12486034 -0.33514531]
Tune response: #8 QCORR_009 [ 0.12493914 -0.33546973]
Tune response: #9 QCORR_010 [ 0.12493914 -0.33546973]
Tune response: #10 QCORR_011 [ 0.12507861 -0.33620251]
Tune response: #11 QCORR_012 [ 0.12507861 -0.33620251]
Tune response: #12 QCORR_013 [ 0.25206877 -0.43354454]
Tune response: #13 QCORR_014 [ 0.12486482 -0.3361889 ]
Tune response: #14 QCORR_015 [ 0.12486482 -0.3361889 ]
Tune response: #15 QCORR_016 [ 0.12488068 -0.33530978]
Tune response: #16 QCORR_017 [ 0.12488068 -0.33530978]
Tune response: #17 QCORR_018 [ 0.23383542 -0.39128504]
Tune response: #18 QCORR_019 [ 0.51255

The correction matrix can be saved to a file (and added to the configuration file).

In [55]:
tune_correction.response.save_json("tune_response_matrix_live.json")
# tune_correction.response.load_json('tune_response_matrix.json')

In [56]:
tune_correction.set([0.21, 0.31], iter=10, wait_time=1.2)

In [57]:
print(tune_correction.readback())
print(tune_correction.get_peer())
tune_correction

[0.21001973 0.30977679]
TangoControlSystem:live


Tune(peer='TangoControlSystem:live', name='DEFAULT_TUNE_CORRECTION', lattice_names=None, quad_array='QCORR', betatron_tune='BETATRON_TUNE', delta=0.001)

In [58]:
tune_monitor.tune.get()

array([0.21001973, 0.30977679])

It is also possible instead of measuring to load the file of a response matrix

In [59]:
# tune_correction.response.load_json('tune_response_matrix_live.json')

In [60]:
print(f"Tune readback is {tune_correction.readback()}")
qx, qy = 0.19, 0.28
print(f"Let's change the tune to {qx=:}, {qy=:}")
tune_correction.set([qx, qy], iter=10, wait_time=1.2)
print(f"Tune readback is {tune_correction.readback()}")

qx, qy = 0.21, 0.3
print(f"Let's change the tune to {qx=:}, {qy=:}")

tune_correction.set([qx, qy], iter=10, wait_time=1.2)
print(f"Tune readback is {tune_correction.readback()}")

# You can try to do something stupid too!
# qx, qy = 0.5, 0.0
# print(f"Let's do something stupid and change the tune to {qx=:}, {qy=:}")
# tune_correction.set([qx, qy])

Tune readback is [0.21001973 0.30977679]
Let's change the tune to qx=0.19, qy=0.28
Tune readback is [0.18668703 0.26823733]
Let's change the tune to qx=0.21, qy=0.3
Tune readback is [0.21768279 0.31389756]


### Manual tune correction

In [32]:
control_mode = sr.live

In [33]:
tunemat = np.zeros((len(qcorrectors), 2))
design_tune = tune_monitor.tune.get()
print(f"Design tune is {design_tune}")
print(f"Tune in live mode is {tune_monitor.tune.get()}")

Design tune is [0.21427229 0.30571746]
Tune in live mode is [0.21427229 0.30571746]


In [34]:
from time import sleep

from tqdm.notebook import tqdm

for idx, m in enumerate(tqdm(qcorrectors)):
    strength = m.strength.get()
    delta = 1e-3
    m.strength.set(strength + delta)
    # print(f"Changing magnet strength in {m.get_name()}")
    sleep(1)
    dq = tune_monitor.tune.get() - design_tune
    tunemat[idx] = dq * delta
    m.strength.set(strength)

# Compute correction matrix
correctionmat = np.linalg.pinv(tunemat.T)

  0%|          | 0/208 [00:00<?, ?it/s]

In [35]:
old_strength_values = qcorrectors.strengths.get()
print(f"Tune before correction {tune_monitor.tune.get()}")
dqx, dqy = 0.01, -0.02
print(f"Chanding tune by [{dqx}, {dqy}]")
qcorrectors.strengths.set(old_strength_values + np.matmul(correctionmat, [dqx, dqy]))
sleep(3)
# tune_monitor.tune.get()
print(f"Tune after correction {tune_monitor.tune.get()}")
qcorrectors.strengths.set(old_strength_values)
sleep(3)
print(
    f"Tune setting the tune back to original value correction {tune_monitor.tune.get()}"
)

Tune before correction [0.21439602 0.30539334]
Chanding tune by [0.01, -0.02]
Tune after correction [nan nan]
Tune setting the tune back to original value correction [0.21427229 0.30571746]
